#**POLI170A Project**

Research question: Is there an association between the availability of alcohol and the rates of particular types of crime in San Diego County?

##Setup

In [61]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
zipcode_df = pd.read_csv("/content/drive/MyDrive/zipcode.csv", header = None)
zipcode_arr = zipcode_df[0].values

### Crime Dataset

In [63]:
import pandas as pd

sdcrime_16_20_allyears_df =  pd.read_csv('http://library.metatab.org/arjis.org-crime_victims-pra-1.1.1/data/sdcrime_16_20_allyears.csv')
sdcrime_16_20_nosusp_df =  pd.read_csv('http://library.metatab.org/arjis.org-crime_victims-pra-1.1.1/data/sdcrime_16_20_nosusp.csv')
sdcrime_16_20_uniquecase_df =  pd.read_csv('http://library.metatab.org/arjis.org-crime_victims-pra-1.1.1/data/sdcrime_16_20_uniquecase.csv')
ucrcodes_df =  pd.read_csv('http://library.metatab.org/arjis.org-crime_victims-pra-1.1.1/data/ucrcodes.csv')

In [64]:
sdcrime_16_20_uniquecase_df.head(3)

,activitynumber,activitydate,year,agency,violationsection,violationtype,chargedescription,chargelevel,codeucr,crimecategory,personrole,race,age,sex,zipcode,censusblock,censustract,city,city.1
0,'01600014',2016-01-01 00:00:00,2016,NATIONAL CITY,10851,VC,TAKE VEHICLE W/O OWNER'S CONSENT/VEHICLE THEFT,FELONY,7A0,Vehicle Theft,VICTIM,OTHER,NaN,FEMALE,91950.0,'02000','022000',NATIONAL CITY,NATIONAL CITY
1,'16000042',2016-01-01 00:00:00,2016,SAN DIEGO,459,PC,BURGLARY/UNSPECIFIED,FELONY,5A6,Non Res Burglary,NaN,NaN,NaN,NaN,92109.0,'20000','007907',SAN DIEGO,SAN DIEGO
2,'16005661',2016-01-01 00:00:00,2016,SAN DIEGO,488,PC,PETTY THEFT,MISDEMEANOR,6DG,Larceny < $400,VICTIM/WITNESS,HISPANIC,74.0,MALE,92115.0,'20230','002902',SAN DIEGO,SAN DIEGO


In [65]:
# drop all rows that containing nan in zipcode
crime_df = sdcrime_16_20_uniquecase_df.dropna(subset=['zipcode'])

# convert zipcode type into int64
crime_df.loc[:, 'zipcode'] = crime_df['zipcode'].astype(int)

# filter df only include sd cities
crime_df = crime_df[crime_df['zipcode'].isin(zipcode_arr)]

In [66]:
crime_df

,activitynumber,activitydate,year,agency,violationsection,violationtype,chargedescription,chargelevel,codeucr,crimecategory,personrole,race,age,sex,zipcode,censusblock,censustract,city,city.1
0,'01600014',2016-01-01 00:00:00,2016,NATIONAL CITY,10851,VC,TAKE VEHICLE W/O OWNER'S CONSENT/VEHICLE THEFT,FELONY,7A0,Vehicle Theft,VICTIM,OTHER,NaN,FEMALE,91950.0,'02000','022000',NATIONAL CITY,NATIONAL CITY
1,'16000042',2016-01-01 00:00:00,2016,SAN DIEGO,459,PC,BURGLARY/UNSPECIFIED,FELONY,5A6,Non Res Burglary,NaN,NaN,NaN,NaN,92109.0,'20000','007907',SAN DIEGO,SAN DIEGO
2,'16005661',2016-01-01 00:00:00,2016,SAN DIEGO,488,PC,PETTY THEFT,MISDEMEANOR,6DG,Larceny < $400,VICTIM/WITNESS,HISPANIC,74.0,MALE,92115.0,'20230','002902',SAN DIEGO,SAN DIEGO
3,'16008822',2016-01-01 00:00:00,2016,SAN DIEGO,487(A),PC,GRAND THEFT:MONEY/LABOR/PROPERTY OVER $950,FELONY,6AE,Larceny >= $400,VICTIM/WITNESS,WHITE,70.0,MALE,92109.0,'20120','007905',SAN DIEGO,SAN DIEGO
5,'16016207',2016-01-01 00:00:00,2016,SAN DIEGO,487,PC,GRAND THEFT/UNSPECIFIED,FELONY,6AG,Larceny >= $400,VICTIM,WHITE,66.0,FEMALE,92126.0,'10040','008355',SAN DIEGO,SAN DIEGO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312084,'20132063',2020-07-31 23:00:00,2020,SHERIFF,10851(A),VC,TAKE VEHICLE W/O OWNER'S CONSENT/VEHICLE THEFT...,FELONY,7A0,Vehicle Theft,VICTIM,HISPANIC,53.0,MALE,91945.0,'02015','014400',LEMON GROVE,LEMON GROVE
312085,'20303289',2020-07-31 23:00:00,2020,SAN DIEGO,10851(A),VC,TAKE VEHICLE W/O OWNER'S CONSENT/VEHICLE THEFT...,FELONY,7A0,Vehicle Theft,VICTIM,OTHER,25.0,MALE,92114.0,'20010','003004',SAN DIEGO,SAN DIEGO
312086,'20303297',2020-07-31 23:00:00,2020,SAN DIEGO,10851(A),VC,TAKE VEHICLE W/O OWNER'S CONSENT/VEHICLE THEFT...,FELONY,7A0,Vehicle Theft,VICTIM,WHITE,28.0,MALE,92108.0,'10110','008902',SAN DIEGO,SAN DIEGO
312087,'20132066',2020-07-31 23:23:00,2020,SHERIFF,245A1,PC,ASSAULT W/DEADLY WEAPON:NOT F/ARM (F),FELONY,4C0,Aggravated Assault,VICTIM,WHITE,31.0,MALE,92028.0,'02000','018906',FALLBROOK,FALLBROOK


### Alcohol Dataset

In [67]:
ca_abc_licenses_df =  pd.read_csv('http://library.metatab.org/abc.ca.gov-licenses-2.1.2/data/ca_abc_licenses.csv')
sd_abc_density_df =  pd.read_csv('http://library.metatab.org/abc.ca.gov-licenses-2.1.2/data/sd_abc_density.csv')

In [69]:
ca_abc_licenses_df.head(3)

,license_type,file_number,lic_or_app,type_status,type_orig_iss_date,expir_date,fee_codes,dup_counts,master_ind,term_in_of_months,...,dba_name,mail_addr_1,mail_addr_2,mail_city,mail_state,mail_zip,prem_county,prem_census_tract,county,tract
0,10.0,596658.0,LIC,ACTIVE,01-MAY-2019,30-APR-2021,P40,,Y,12.0,...,BROOKLYN BREWERY,79 N 11TH ST,,BROOKLYN,NY,11249,ALAMEDA,4324.00,1.0,14000US06001432400
1,10.0,604941.0,LIC,ACTIVE,10-MAY-2019,28-FEB-2021,P0,,Y,12.0,...,SAMANDLA WINE IMPORTS,7582 S SAULSBURY CT,,LITTLETON,CO,80128,SONOMA,1501.00,97.0,14000US06097150100
2,10.0,602586.0,LIC,ACTIVE,15-MAY-2019,30-APR-2021,P40,,Y,12.0,...,JIM BEAM BRANDS CO,222 MERCHANDISE MART PLZ,STE 1600,CHICAGO,IL,60654-4262,SAN FRANCISCO,0615.00,75.0,14000US06075061500


In [72]:
# limit the area only San Diego, CA
sd_alcohol = ca_abc_licenses_df[(ca_abc_licenses_df['mail_state'] == "CA") & (ca_abc_licenses_df['prem_county'] == "SAN DIEGO")]

# limit license_type only 41 and 47
onsale_df = sd_alcohol[((sd_alcohol['license_type'] == 47) | (sd_alcohol['license_type'] == 41)) & (sd_alcohol['type_status'] == "ACTIVE")].reset_index(drop = True)

In [74]:
onsale_df.head(3)

,license_type,file_number,lic_or_app,type_status,type_orig_iss_date,expir_date,fee_codes,dup_counts,master_ind,term_in_of_months,...,dba_name,mail_addr_1,mail_addr_2,mail_city,mail_state,mail_zip,prem_county,prem_census_tract,county,tract
0,47.0,328604.0,APP,ACTIVE,,,P40,003,N,12.0,...,PARK HYATT AVIARA,7100 AVIARA RESORT DR,,CARLSBAD,CA,92011-4908,SAN DIEGO,0178.08,73.0,14000US06073017808
1,47.0,344010.0,APP,ACTIVE,,,P40,001,N,12.0,...,VISTA ENTERTAINMENT CENTER,380 S MELROSE DR,STE 300,VISTA,CA,92081,SAN DIEGO,0195.03,73.0,14000US06073019503
2,47.0,380888.0,APP,ACTIVE,,,P40,002,N,12.0,...,HOUSE OF BLUES,9348 CIVIC CENTER DR,,BEVERLY HILLS,CA,90210-3624,SAN DIEGO,0053.00,73.0,14000US06073005300


In [77]:
onsale_df.groupby('mail_city')['file_number'].count().mean()

15.727272727272727

In [75]:
onsale_df['mail_city'].unique()

array(['CARLSBAD', 'VISTA', 'BEVERLY HILLS', 'CORTE MADERA',
       'SAN FRANCISCO', 'SAN DIEGO', 'EL CAJON', 'LA JOLLA', 'LA MESA',
       'RANCHO SANTA FE', 'CHULA VISTA', 'VAN NUYS', 'PLAYA VISTA',
       'NEWPORT BEACH', 'ESCONDIDO', 'WEST HOLLYWOOD', 'SHERMAN OAKS',
       'POWAY', 'JAMUL', 'EL SEGUNDO', 'SOLANA BEACH', 'BORREGO SPRINGS',
       'HUNTINGTON BEACH', 'JACUMBA', 'ALPINE', 'VALLEY CENTER', 'JULIAN',
       'SAN RAFAEL', 'FALLBROOK', 'OCEANSIDE', 'COSTA MESA', 'LAKESIDE',
       'PAUMA VALLEY', 'CALABASAS', 'CORONADO', 'PALA', 'SANTEE',
       'IMPERIAL BEACH', 'CARDIFF', 'LOS ANGELES', 'NATIONAL CITY',
       'IRVINE', 'DEL MAR', 'REDONDO BEACH', 'ENCINITAS', 'SAUSALITO',
       'BONITA', 'SAN MARCOS', 'TEMECULA', 'FREMONT', 'BRISBANE',
       'SANTA ANA', 'HAWTHORNE', 'WESTLAKE VILLAGE', 'PASADENA',
       'CULVER CITY', 'LEMON GROVE', 'ALTA LOMA', 'SPRING VALLEY',
       'SOUTH EL MONTE', 'ARCADIA', 'CANYON COUNTRY', 'CYPRESS', 'ENCINO',
       'ROSEMEAD', 'DANA POI